<a href="https://colab.research.google.com/github/xin-kai08/Machine-Learning-Models/blob/main/machine_learning_f3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

# 掛載 Google 雲端硬碟
drive.mount('/content/drive')

# 資料集根目錄
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/dataset/feature dim_3"

# 各分類資料夾設定
LABEL_DIRS = {
    0: os.path.join(BASE_PATH, "normal"),
    1: os.path.join(BASE_PATH, "abnormal/transformer_rust"),
    2: os.path.join(BASE_PATH, "abnormal/wire_rust"),
    3: os.path.join(BASE_PATH, "abnormal/wire_peeling"),
}

Mounted at /content/drive


定義快取函數(三維)

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def generate_preprocessed_cache_3d(seq_lens, label_dirs, cache_dir="/content/preprocessed"):
    os.makedirs(cache_dir, exist_ok=True)

    for seq_len in seq_lens:
        cache_X = os.path.join(cache_dir, f"X_seq{seq_len}_3d.npy")
        cache_y = os.path.join(cache_dir, f"y_seq{seq_len}_3d.npy")

        if os.path.exists(cache_X) and os.path.exists(cache_y):
            print(f"📥 已存在 3D 快取：seq_len={seq_len}，略過")
            continue

        all_seq, all_labels = [], []
        for label, folder in label_dirs.items():
            for fname in os.listdir(folder):
                if fname.endswith(".csv"):
                    path = os.path.join(folder, fname)
                    df = pd.read_csv(path)
                    data = df[["voltage", "current", "power"]].values.astype(np.float32)
                    num_chunks = len(data) // seq_len
                    chunks = [data[i * seq_len : (i + 1) * seq_len] for i in range(num_chunks)]
                    all_seq.extend(chunks)
                    all_labels.extend([label] * len(chunks))

        seq_arr = np.array(all_seq, dtype=np.float32)
        labels_arr = np.array(all_labels, dtype=np.int64)
        B, T, F = seq_arr.shape
        reshaped = seq_arr.reshape(-1, F)
        scaled = StandardScaler().fit_transform(reshaped).reshape(B, T, F)

        np.save(cache_X, scaled)
        np.save(cache_y, labels_arr)
        print(f"✅ 完成 3D 快取：seq_len={seq_len}（共 {B} 筆序列）")

定義快取函數(二維)

In [3]:
def generate_preprocessed_cache_2d(seq_lens, label_dirs, cache_dir="/content/preprocessed"):
    os.makedirs(cache_dir, exist_ok=True)

    for seq_len in seq_lens:
        cache_X = os.path.join(cache_dir, f"X_seq{seq_len}_2d.npy")
        cache_y = os.path.join(cache_dir, f"y_seq{seq_len}_2d.npy")

        if os.path.exists(cache_X) and os.path.exists(cache_y):
            print(f"📥 已存在 2D 快取：seq_len={seq_len}，略過")
            continue

        all_features, all_labels = [], []
        for label, folder in label_dirs.items():
            for fname in os.listdir(folder):
                if fname.endswith(".csv"):
                    path = os.path.join(folder, fname)
                    df = pd.read_csv(path)
                    data = df[["voltage", "current", "power"]].values.astype(np.float32)
                    num_chunks = len(data) // seq_len
                    chunks = [data[i * seq_len : (i + 1) * seq_len] for i in range(num_chunks)]

                    for chunk in chunks:
                        features = []
                        features.extend(np.mean(chunk, axis=0))  # 3
                        features.extend(np.std(chunk, axis=0))   # 3
                        features.extend(np.max(chunk, axis=0))   # 3
                        features.extend(np.min(chunk, axis=0))   # 3
                        all_features.append(features)
                        all_labels.append(label)

        X = np.array(all_features, dtype=np.float32)
        y = np.array(all_labels, dtype=np.int64)

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        np.save(cache_X, X_scaled)
        np.save(cache_y, y)
        print(f"✅ 完成 2D 快取：seq_len={seq_len}（共 {len(X_scaled)} 筆）")

實際執行快取

In [5]:
seq_lens = [4, 5, 8, 10]
generate_preprocessed_cache_3d(seq_lens, LABEL_DIRS)      # 產生 3D
generate_preprocessed_cache_2d(seq_lens, LABEL_DIRS)   # 產生 2D

✅ 完成 3D 快取：seq_len=4（共 10116 筆序列）
✅ 完成 3D 快取：seq_len=5（共 8101 筆序列）
✅ 完成 3D 快取：seq_len=8（共 5047 筆序列）
✅ 完成 3D 快取：seq_len=10（共 4047 筆序列）
✅ 完成 2D 快取：seq_len=4（共 10116 筆）
✅ 完成 2D 快取：seq_len=5（共 8101 筆）
✅ 完成 2D 快取：seq_len=8（共 5047 筆）
✅ 完成 2D 快取：seq_len=10（共 4047 筆）


LSTM

In [6]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# === 資料夾與儲存路徑設定 ===
RESULT_DIR = "/content/drive/MyDrive/Colab Notebooks/test/feature dim_3/LSTM"
os.makedirs(RESULT_DIR, exist_ok=True)

SUB_DIRS = [
    "accuracy_curves",
    "loss_curves",
    "f1_score_curves",
    "confusion_matrices",
    "train_accuracy",
    "train_loss",
]

for sub in SUB_DIRS:
    os.makedirs(os.path.join(RESULT_DIR, sub), exist_ok=True)

# === 自訂 Dataset ===
class ChargeSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# === LSTM 模型 ===
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

# === 主訓練與超參數搜尋函數（使用快取） ===
def train_and_search_lstm(batch_sizes, learning_rates, seq_lens,
                          num_epochs=100, hidden_dim=64, num_layers=1,
                          num_classes=4, k_folds=5):

    PREPROCESSED_DIR = "/content/preprocessed/"

    results = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    for bs in batch_sizes:
        for lr in learning_rates:
            for seq_len in seq_lens:
                print(f"\n🧪 BS={bs} | LR={lr} | SEQ={seq_len}")
                x_path = os.path.join(PREPROCESSED_DIR, f"X_seq{seq_len}_3d.npy")
                y_path = os.path.join(PREPROCESSED_DIR, f"y_seq{seq_len}_3d.npy")
                X = np.load(x_path)
                y = np.load(y_path)

                skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

                acc_curves, loss_curves, f1_curves = [], [], []
                acc_train_curves, loss_train_curves = [], []
                all_y_true, all_y_pred = [], []

                t0 = time.time()

                for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                    X_train, X_val = X[train_idx], X[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]

                    train_loader = DataLoader(ChargeSequenceDataset(X_train, y_train), batch_size=bs, shuffle=True)
                    val_loader = DataLoader(ChargeSequenceDataset(X_val, y_val), batch_size=bs)

                    model = LSTMClassifier(input_dim=3, hidden_dim=hidden_dim,
                                           num_layers=num_layers, num_classes=num_classes).to(device)
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                    criterion = nn.CrossEntropyLoss()

                    acc_list, loss_list, f1_list = [], [], []
                    acc_train_list, loss_train_list = [], []

                    for epoch in range(num_epochs):
                        # === Training ===
                        model.train()
                        total_loss_train, correct_train, total_train = 0, 0, 0
                        for xb, yb in train_loader:
                            xb, yb = xb.to(device), yb.to(device)
                            optimizer.zero_grad()
                            out = model(xb)
                            loss = criterion(out, yb)
                            loss.backward()
                            optimizer.step()

                            _, pred = torch.max(out, 1)
                            correct_train += (pred == yb).sum().item()
                            total_train += yb.size(0)
                            total_loss_train += loss.item() * xb.size(0)

                        acc_train = correct_train / total_train
                        loss_train = total_loss_train / total_train

                        acc_train_list.append(acc_train)
                        loss_train_list.append(loss_train)

                        # === Validation ===
                        model.eval()
                        correct, total, total_loss = 0, 0, 0
                        y_pred_epoch, y_true_epoch = [], []
                        with torch.no_grad():
                            for xb, yb in val_loader:
                                xb, yb = xb.to(device), yb.to(device)
                                out = model(xb)
                                loss = criterion(out, yb)
                                _, pred = torch.max(out, 1)
                                correct += (pred == yb).sum().item()
                                total += yb.size(0)
                                total_loss += loss.item() * xb.size(0)
                                y_pred_epoch.extend(pred.cpu().numpy())
                                y_true_epoch.extend(yb.cpu().numpy())

                        acc = correct / total
                        avg_loss = total_loss / total
                        f1 = f1_score(y_true_epoch, y_pred_epoch, average='macro', zero_division=0)

                        acc_list.append(acc)
                        loss_list.append(avg_loss)
                        f1_list.append(f1)

                        if (epoch + 1) % 10 == 0:
                            print(f"    [Fold {fold+1}] Epoch {epoch+1}/{num_epochs} | Acc: {acc:.4f} | Loss: {avg_loss:.4f}")

                    acc_curves.append(acc_list)
                    loss_curves.append(loss_list)
                    f1_curves.append(f1_list)
                    acc_train_curves.append(acc_train_list)
                    loss_train_curves.append(loss_train_list)
                    all_y_true.extend(y_true_epoch)
                    all_y_pred.extend(y_pred_epoch)

                t1 = time.time()

                def plot_metric(avg_list, metric_name, color, marker, folder):
                    plt.plot(range(1, num_epochs+1), avg_list, marker=marker, color=color)
                    plt.title(f"{metric_name.capitalize()} (BS={bs}, LR={lr}, SEQ={seq_len})")
                    plt.xlabel("Epoch")
                    plt.ylabel(metric_name.capitalize())
                    plt.ylim(0, 1.0 if metric_name != 'loss' else None)
                    plt.grid(True)
                    path = os.path.join(RESULT_DIR, folder, f"bs{bs}_lr{lr}_seq{seq_len}.png")
                    plt.savefig(path, bbox_inches='tight')
                    plt.close()

                # === Plot ===
                plot_metric(np.mean(acc_curves, axis=0), "accuracy", "black", "o", "accuracy_curves")
                plot_metric(np.mean(loss_curves, axis=0), "loss", "orange", "s", "loss_curves")
                plot_metric(np.mean(f1_curves, axis=0), "f1_score", "purple", "^", "f1_score_curves")
                plot_metric(np.mean(acc_train_curves, axis=0), "accuracy", "red", "o", "train_accuracy")
                plot_metric(np.mean(loss_train_curves, axis=0), "loss", "blue", "s", "train_loss")

                # 混淆矩陣
                cm = confusion_matrix(all_y_true, all_y_pred)
                disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
                disp.plot(cmap='Blues', values_format='d')
                plt.title(f"Confusion Matrix\nBS={bs} LR={lr} SEQ={seq_len}")
                plt.savefig(os.path.join(RESULT_DIR, "confusion_matrices", f"bs{bs}_lr{lr}_seq{seq_len}.png"), bbox_inches='tight')
                plt.close()

                final_result = {
                    'batch_size': bs, 'learning_rate': lr, 'seq_len': seq_len,
                    'final_acc': acc_list[-1], 'final_loss': loss_list[-1],
                    'precision': precision_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'recall': recall_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'f1_score': f1_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'training_time_s': round(t1 - t0, 2)
                }
                results.append(final_result)

                print(f"\n📊 統計結果 (BS={bs}, LR={lr}, SEQ={seq_len}):")
                print(f"  🔹 Final Accuracy : {final_result['final_acc']:.4f}")
                print(f"  🔹 F1-score       : {final_result['f1_score']:.4f}")
                print(f"  ⏱️  Training Time  : {final_result['training_time_s']} 秒")

    df = pd.DataFrame(results)
    csv_path = os.path.join(RESULT_DIR, "lstm_experiment_results.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n📄 All experiment results saved to {csv_path}")

    if not df.empty:
        best = df.loc[df['final_acc'].idxmax()]
        print(f"\n🏆 Best: BS={best['batch_size']} | LR={best['learning_rate']} | SEQ={best['seq_len']} | ACC={best['final_acc']:.4f}")
        return results, best
    else:
        print("\n❗ No valid results")
        return [], None

In [7]:
# 設定訓練參數
batch_sizes = [4, 8, 16]
learning_rates = [1e-3, 1e-4]
seq_lens = [4, 5, 8, 10]
num_epochs = 100

# 顯示總組數
total_combinations = len(batch_sizes) * len(learning_rates) * len(seq_lens)
print(f"🔍 總共訓練組數：{total_combinations}，每組訓練 {num_epochs} epochs")

# 執行網格搜尋訓練
results, best = train_and_search_lstm(batch_sizes, learning_rates, seq_lens, num_epochs=num_epochs)

# 輸出最佳參數與指標
if best is not None:
    print("\n🎯 最佳參數組合：")
    print(f"Batch Size     = {best['batch_size']}")
    print(f"Learning Rate  = {best['learning_rate']}")
    print(f"Sequence Length= {best['seq_len']}")
    print(f"Final Accuracy = {best['final_acc']:.4f}")
    print(f"Final Loss     = {best['final_loss']:.4f}")#
    print(f"Precision      = {best['precision']:.4f}")
    print(f"Recall         = {best['recall']:.4f}")
    print(f"F1-score       = {best['f1_score']:.4f}")
    print(f"Training Time  = {best['training_time_s']} 秒")
else:
    print("❗ 沒有有效結果（準確率全部為 1.0）")

🔍 總共訓練組數：1，每組訓練 50 epochs

🧪 BS=16 | LR=0.001 | SEQ=10
    [Fold 1] Epoch 10/50 | Acc: 0.9753 | Loss: 0.0885
    [Fold 1] Epoch 20/50 | Acc: 0.9852 | Loss: 0.0474
    [Fold 1] Epoch 30/50 | Acc: 0.9938 | Loss: 0.0330
    [Fold 1] Epoch 40/50 | Acc: 0.9901 | Loss: 0.0400
    [Fold 1] Epoch 50/50 | Acc: 0.9951 | Loss: 0.0378
    [Fold 2] Epoch 10/50 | Acc: 0.9815 | Loss: 0.0581
    [Fold 2] Epoch 20/50 | Acc: 0.9864 | Loss: 0.0446
    [Fold 2] Epoch 30/50 | Acc: 0.9901 | Loss: 0.0277
    [Fold 2] Epoch 40/50 | Acc: 0.9741 | Loss: 0.0652
    [Fold 2] Epoch 50/50 | Acc: 0.9926 | Loss: 0.0209
    [Fold 3] Epoch 10/50 | Acc: 0.9778 | Loss: 0.0881
    [Fold 3] Epoch 20/50 | Acc: 0.9827 | Loss: 0.0592
    [Fold 3] Epoch 30/50 | Acc: 0.9827 | Loss: 0.0755
    [Fold 3] Epoch 40/50 | Acc: 0.9839 | Loss: 0.0712
    [Fold 3] Epoch 50/50 | Acc: 0.9889 | Loss: 0.0619
    [Fold 4] Epoch 10/50 | Acc: 0.9802 | Loss: 0.0551
    [Fold 4] Epoch 20/50 | Acc: 0.9876 | Loss: 0.0579
    [Fold 4] Epoch 30/50 | 

MLP

In [15]:
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# === 資料夾與儲存路徑設定 ===
RESULT_DIR = "/content/drive/MyDrive/Colab Notebooks/test/feature dim_3/MLP"
os.makedirs(RESULT_DIR, exist_ok=True)

SUB_DIRS = [
    "accuracy_curves",
    "loss_curves",
    "f1_score_curves",
    "confusion_matrices",
    "train_accuracy",
    "train_loss",
]

for sub in SUB_DIRS:
    os.makedirs(os.path.join(RESULT_DIR, sub), exist_ok=True)

# === 自訂 Dataset ===
class ChargeSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# === MLP 模型 ===
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )
    def forward(self, x):
        return self.model(x)

# === 訓練與儲存 ===
def train_and_search_mlp(batch_sizes, learning_rates, seq_lens,
                         num_epochs=100, hidden_dim=128,
                         num_classes=4, k_folds=5):

    PREPROCESSED_DIR = "./preprocessed/"

    results = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    for bs in batch_sizes:
        for lr in learning_rates:
            for seq_len in seq_lens:
                print(f"\n🧪 BS={bs} | LR={lr} | SEQ={seq_len}")
                x_path = os.path.join(PREPROCESSED_DIR, f"X_seq{seq_len}_2d.npy")
                y_path = os.path.join(PREPROCESSED_DIR, f"y_seq{seq_len}_2d.npy")
                X = np.load(x_path)
                y = np.load(y_path)
                input_dim = X.shape[1]

                skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

                acc_curves, loss_curves, f1_curves = [], [], []
                acc_train_curves, loss_train_curves = [], []
                all_y_true, all_y_pred = [], []

                t0 = time.time()

                for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                    X_train, X_val = X[train_idx], X[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]

                    train_loader = DataLoader(ChargeSequenceDataset(X_train, y_train), batch_size=bs, shuffle=True)
                    val_loader = DataLoader(ChargeSequenceDataset(X_val, y_val), batch_size=bs)

                    model = MLPClassifier(input_dim=input_dim, hidden_dim=hidden_dim, num_classes=num_classes).to(device)
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                    criterion = nn.CrossEntropyLoss()

                    acc_list, loss_list, f1_list = [], [], []
                    acc_train_list, loss_train_list = [], []

                    for epoch in range(num_epochs):
                        # === Training ===
                        model.train()
                        total_loss_train, correct_train, total_train = 0, 0, 0
                        for xb, yb in train_loader:
                            xb, yb = xb.to(device), yb.to(device)
                            optimizer.zero_grad()
                            out = model(xb)
                            loss = criterion(out, yb)
                            loss.backward()
                            optimizer.step()

                            _, pred = torch.max(out, 1)
                            correct_train += (pred == yb).sum().item()
                            total_train += yb.size(0)
                            total_loss_train += loss.item() * xb.size(0)

                        acc_train = correct_train / total_train
                        loss_train = total_loss_train / total_train

                        acc_train_list.append(acc_train)
                        loss_train_list.append(loss_train)

                        # === Validation ===
                        model.eval()
                        correct, total, total_loss = 0, 0, 0
                        y_pred_epoch, y_true_epoch = [], []
                        with torch.no_grad():
                            for xb, yb in val_loader:
                                xb, yb = xb.to(device), yb.to(device)
                                out = model(xb)
                                loss = criterion(out, yb)
                                _, pred = torch.max(out, 1)
                                correct += (pred == yb).sum().item()
                                total += yb.size(0)
                                total_loss += loss.item() * xb.size(0)
                                y_pred_epoch.extend(pred.cpu().numpy())
                                y_true_epoch.extend(yb.cpu().numpy())

                        acc = correct / total
                        avg_loss = total_loss / total
                        f1 = f1_score(y_true_epoch, y_pred_epoch, average='macro', zero_division=0)

                        acc_list.append(acc)
                        loss_list.append(avg_loss)
                        f1_list.append(f1)

                        if (epoch + 1) % 10 == 0:
                            print(f"    [Fold {fold+1}] Epoch {epoch+1}/{num_epochs} | Acc: {acc:.4f} | Loss: {avg_loss:.4f}")

                    acc_curves.append(acc_list)
                    loss_curves.append(loss_list)
                    f1_curves.append(f1_list)
                    acc_train_curves.append(acc_train_list)
                    loss_train_curves.append(loss_train_list)
                    all_y_true.extend(y_true_epoch)
                    all_y_pred.extend(y_pred_epoch)

                t1 = time.time()

                def plot_metric(avg_list, metric_name, color, marker, folder):
                    plt.plot(range(1, num_epochs+1), avg_list, marker=marker, color=color)
                    plt.title(f"{metric_name.capitalize()} (BS={bs}, LR={lr}, SEQ={seq_len})")
                    plt.xlabel("Epoch")
                    plt.ylabel(metric_name.capitalize())
                    plt.ylim(0, 1.0 if metric_name != 'loss' else None)
                    plt.grid(True)
                    path = os.path.join(RESULT_DIR, folder, f"bs{bs}_lr{lr}_seq{seq_len}.png")
                    plt.savefig(path, bbox_inches='tight')
                    plt.close()

                # === Plot ===
                plot_metric(np.mean(acc_curves, axis=0), "accuracy", "black", "o", "accuracy_curves")
                plot_metric(np.mean(loss_curves, axis=0), "loss", "orange", "s", "loss_curves")
                plot_metric(np.mean(f1_curves, axis=0), "f1_score", "purple", "^", "f1_score_curves")
                plot_metric(np.mean(acc_train_curves, axis=0), "accuracy", "red", "o", "train_accuracy")
                plot_metric(np.mean(loss_train_curves, axis=0), "loss", "blue", "s", "train_loss")

                # 混淆矩陣
                cm = confusion_matrix(all_y_true, all_y_pred)
                disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
                disp.plot(cmap='Blues', values_format='d')
                plt.title(f"Confusion Matrix\nBS={bs} LR={lr} SEQ={seq_len}")
                plt.savefig(os.path.join(RESULT_DIR, "confusion_matrices", f"bs{bs}_lr{lr}_seq{seq_len}.png"), bbox_inches='tight')
                plt.close()

                final_result = {
                    'batch_size': bs, 'learning_rate': lr, 'seq_len': seq_len,
                    'final_acc': acc_list[-1], 'final_loss': loss_list[-1],
                    'precision': precision_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'recall': recall_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'f1_score': f1_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'training_time_s': round(t1 - t0, 2)
                }
                results.append(final_result)

                print(f"\n📊 統計結果 (BS={bs}, LR={lr}, SEQ={seq_len}):")
                print(f"  🔹 Final Accuracy : {final_result['final_acc']:.4f}")
                print(f"  🔹 F1-score       : {final_result['f1_score']:.4f}")
                print(f"  ⏱️  Training Time  : {final_result['training_time_s']} 秒")

    df = pd.DataFrame(results)
    csv_path = os.path.join(RESULT_DIR, "mlp_experiment_results.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n📄 All experiment results saved to {csv_path}")

    if not df.empty:
        best = df.loc[df['final_acc'].idxmax()]
        print(f"\n🏆 Best: BS={best['batch_size']} | LR={best['learning_rate']} | SEQ={best['seq_len']} | ACC={best['final_acc']:.4f}")
        return results, best
    else:
        print("\n❗ No valid results")
        return [], None

In [12]:
# 設定訓練參數
batch_sizes = [4, 8, 16]
learning_rates = [1e-3, 1e-4]
seq_lens = [4, 5, 8, 10]
num_epochs = 100

# 顯示總組數
total_combinations = len(batch_sizes) * len(learning_rates) * len(seq_lens)
print(f"🔍 總共訓練組數：{total_combinations}，每組訓練 {num_epochs} epochs")

# 執行網格搜尋訓練
results, best = train_and_search_mlp(batch_sizes, learning_rates, seq_lens, num_epochs=num_epochs)

# 輸出最佳參數與指標
if best is not None:
    print("\n🎯 最佳參數組合：")
    print(f"Batch Size     = {best['batch_size']}")
    print(f"Learning Rate  = {best['learning_rate']}")
    print(f"Sequence Length= {best['seq_len']}")
    print(f"Final Accuracy = {best['final_acc']:.4f}")
    print(f"Final Loss     = {best['final_loss']:.4f}")
    print(f"Precision      = {best['precision']:.4f}")
    print(f"Recall         = {best['recall']:.4f}")
    print(f"F1-score       = {best['f1_score']:.4f}")
    print(f"Training Time  = {best['training_time_s']} 秒")
else:
    print("❗ 沒有有效結果（準確率全部為 1.0）")


🔍 總共訓練組數：1，每組訓練 50 epochs

🧪 BS=16 | LR=0.001 | SEQ=10
    [Fold 1] Epoch 10/50 | Acc: 0.9728 | Loss: 0.1296
    [Fold 1] Epoch 20/50 | Acc: 0.9790 | Loss: 0.0771
    [Fold 1] Epoch 30/50 | Acc: 0.9802 | Loss: 0.0726
    [Fold 1] Epoch 40/50 | Acc: 0.9815 | Loss: 0.0558
    [Fold 1] Epoch 50/50 | Acc: 0.9852 | Loss: 0.0618
    [Fold 2] Epoch 10/50 | Acc: 0.9765 | Loss: 0.1051
    [Fold 2] Epoch 20/50 | Acc: 0.9840 | Loss: 0.0677
    [Fold 2] Epoch 30/50 | Acc: 0.9815 | Loss: 0.0660
    [Fold 2] Epoch 40/50 | Acc: 0.9815 | Loss: 0.0666
    [Fold 2] Epoch 50/50 | Acc: 0.9852 | Loss: 0.0650
    [Fold 3] Epoch 10/50 | Acc: 0.9790 | Loss: 0.0998
    [Fold 3] Epoch 20/50 | Acc: 0.9827 | Loss: 0.0683
    [Fold 3] Epoch 30/50 | Acc: 0.9839 | Loss: 0.0605
    [Fold 3] Epoch 40/50 | Acc: 0.9864 | Loss: 0.0573
    [Fold 3] Epoch 50/50 | Acc: 0.9839 | Loss: 0.0661
    [Fold 4] Epoch 10/50 | Acc: 0.9740 | Loss: 0.0996
    [Fold 4] Epoch 20/50 | Acc: 0.9852 | Loss: 0.0568
    [Fold 4] Epoch 30/50 | 

SVM

In [16]:
# === SVM 訓練程式（改寫為與 LSTM 架構一致）===
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

# === 資料夾與儲存路徑設定 ===
RESULT_DIR = "/content/drive/MyDrive/Colab Notebooks/test/feature dim_3/SVM"
os.makedirs(RESULT_DIR, exist_ok=True)

SUB_DIRS = [
    "accuracy_curves",
    "f1_score_curves",
    "confusion_matrices",
    "train_accuracy",
]

for sub in SUB_DIRS:
    os.makedirs(os.path.join(RESULT_DIR, sub), exist_ok=True)

# === 改寫後的 SVM 訓練流程 ===
def train_svm_search(kernels, Cs, seq_lens, k_folds=5):

    PREPROCESSED_DIR = "./preprocessed/"
    results = []
    best = None

    for kernel in kernels:
        for C_val in Cs:
            for seq_len in seq_lens:
                print(f"\n🔎 Kernel={kernel} | C={C_val} | SEQ={seq_len}")
                x_path = os.path.join(PREPROCESSED_DIR, f"X_seq{seq_len}_2d.npy")
                y_path = os.path.join(PREPROCESSED_DIR, f"y_seq{seq_len}_2d.npy")
                X = np.load(x_path)
                y = np.load(y_path)

                skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
                acc_list, f1_list, all_y_true, all_y_pred = [], [], [], []
                t0 = time.time()

                for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                    X_train, X_val = X[train_idx], X[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]
                    clf = SVC(kernel=kernel, C=C_val)
                    clf.fit(X_train, y_train)
                    y_pred = clf.predict(X_val)
                    acc = accuracy_score(y_val, y_pred)
                    f1 = f1_score(y_val, y_pred, average='macro', zero_division=0)
                    acc_list.append(acc)
                    f1_list.append(f1)
                    all_y_true.extend(y_val)
                    all_y_pred.extend(y_pred)
                t1 = time.time()

                final_acc = np.mean(acc_list)
                final_f1 = np.mean(f1_list)
                if final_acc >= 1.0:
                    print(f"⚠️ Skipped Kernel={kernel} C={C_val} SEQ={seq_len} due to acc=1.0")
                    continue

                precision = precision_score(all_y_true, all_y_pred, average='macro', zero_division=0)
                recall = recall_score(all_y_true, all_y_pred, average='macro', zero_division=0)

                # 新增三種圖
                # 1. K-fold validation accuracy 曲線
                plt.figure()
                plt.plot(range(1, k_folds+1), acc_list, marker='o', label='Validation Accuracy')
                plt.title(f"K-fold Accuracy\nKernel={kernel} C={C_val} SEQ={seq_len}")
                plt.xlabel("Fold")
                plt.ylabel("Accuracy")
                plt.ylim(0, 1.01)
                plt.grid(True)
                plt.savefig(os.path.join(RESULT_DIR, "accuracy_curves", f"k{kernel}_c{C_val}_seq{seq_len}.png"))
                plt.close()
                # 2. K-fold F1-score 曲線
                plt.figure()
                plt.plot(range(1, k_folds+1), f1_list, marker='o', label='Validation F1-score', color='orange')
                plt.title(f"K-fold F1-score\nKernel={kernel} C={C_val} SEQ={seq_len}")
                plt.xlabel("Fold")
                plt.ylabel("F1-score")
                plt.ylim(0, 1.01)
                plt.grid(True)
                plt.savefig(os.path.join(RESULT_DIR, "f1_score_curves", f"k{kernel}_c{C_val}_seq{seq_len}.png"))
                plt.close()
                # 3. 訓練集準確率（整體）
                clf_full = SVC(kernel=kernel, C=C_val)
                clf_full.fit(X, y)
                train_pred = clf_full.predict(X)
                train_acc = accuracy_score(y, train_pred)
                plt.figure()
                plt.bar(['train'], [train_acc])
                plt.title(f"Train Accuracy\nKernel={kernel} C={C_val} SEQ={seq_len}")
                plt.ylim(0, 1.01)
                plt.ylabel("Accuracy")
                plt.savefig(os.path.join(RESULT_DIR, "train_accuracy", f"k{kernel}_c{C_val}_seq{seq_len}.png"))
                plt.close()

                # 混淆矩陣
                cm = confusion_matrix(all_y_true, all_y_pred)
                disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
                disp.plot(cmap='Blues', values_format='d')
                plt.title(f"Confusion Matrix\nKernel={kernel} C={C_val} SEQ={seq_len}")
                plt.savefig(os.path.join(RESULT_DIR, "confusion_matrices", f"k{kernel}_c{C_val}_seq{seq_len}.png"), bbox_inches='tight')
                plt.close()

                result = {
                    'kernel': kernel, 'C': C_val, 'seq_len': seq_len,
                    'final_acc': final_acc, 'precision': precision,
                    'recall': recall, 'f1_score': final_f1,
                    'training_time_s': round(t1 - t0, 2)
                }
                results.append(result)
                if best is None or result['final_acc'] > best['final_acc']:
                    best = result
                print(f"\n📊 統計結果 (Kernel={kernel}, C={C_val}, SEQ={seq_len}):")
                print(f"  🔹 Final Accuracy : {final_acc:.4f}")
                print(f"  🔹 Precision      : {precision:.4f}")
                print(f"  🔹 Recall         : {recall:.4f}")
                print(f"  🔹 F1-score       : {final_f1:.4f}")
                print(f"  ⏱️  Training Time  : {round(t1 - t0, 2)} 秒")

    df = pd.DataFrame(results)
    csv_path = os.path.join(RESULT_DIR, "svm_experiment_results.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n📄 Results saved to {csv_path}")
    return results, best

In [17]:
seq_lens = [4, 8, 10]
C_list = [1.0, 10.0]
kernel_list = ["rbf", "linear"]

results, best = train_svm_search(kernel_list, C_list, seq_lens)

print("\n🎯 最佳參數組合：")
print(f"Sequence Length = {best['seq_len']}")
print(f"C = {best['C']}")
print(f"Kernel = {best['kernel']}")
print(f"Accuracy = {best['final_acc']:.4f}")
print(f"Precision = {best['precision']:.4f}")
print(f"Recall = {best['recall']:.4f}")
print(f"F1-score = {best['f1_score']:.4f}")
print(f"Training Time = {best['training_time_s']} 秒")


🔎 Kernel=rbf | C=1.0 | SEQ=4

📊 統計結果 (Kernel=rbf, C=1.0, SEQ=4):
  🔹 Final Accuracy : 0.9807
  🔹 Precision      : 0.9729
  🔹 Recall         : 0.9808
  🔹 F1-score       : 0.9768
  ⏱️  Training Time  : 2.0 秒

🔎 Kernel=rbf | C=1.0 | SEQ=8

📊 統計結果 (Kernel=rbf, C=1.0, SEQ=8):
  🔹 Final Accuracy : 0.9816
  🔹 Precision      : 0.9717
  🔹 Recall         : 0.9820
  🔹 F1-score       : 0.9767
  ⏱️  Training Time  : 0.59 秒

🔎 Kernel=rbf | C=1.0 | SEQ=10

📊 統計結果 (Kernel=rbf, C=1.0, SEQ=10):
  🔹 Final Accuracy : 0.9815
  🔹 Precision      : 0.9723
  🔹 Recall         : 0.9799
  🔹 F1-score       : 0.9761
  ⏱️  Training Time  : 0.41 秒

🔎 Kernel=rbf | C=10.0 | SEQ=4

📊 統計結果 (Kernel=rbf, C=10.0, SEQ=4):
  🔹 Final Accuracy : 0.9870
  🔹 Precision      : 0.9811
  🔹 Recall         : 0.9851
  🔹 F1-score       : 0.9831
  ⏱️  Training Time  : 1.55 秒

🔎 Kernel=rbf | C=10.0 | SEQ=8

📊 統計結果 (Kernel=rbf, C=10.0, SEQ=8):
  🔹 Final Accuracy : 0.9885
  🔹 Precision      : 0.9826
  🔹 Recall         : 0.9865
  🔹 F1-score 

GRU

In [20]:
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# === 資料夾與儲存路徑設定 ===
RESULT_DIR = "/content/drive/MyDrive/Colab Notebooks/test/feature dim_3/GRU"
os.makedirs(RESULT_DIR, exist_ok=True)

SUB_DIRS = [
    "accuracy_curves",
    "loss_curves",
    "f1_score_curves",
    "confusion_matrices",
    "train_accuracy",
    "train_loss",
]

for sub in SUB_DIRS:
    os.makedirs(os.path.join(RESULT_DIR, sub), exist_ok=True)

# === 自訂 Dataset ===
class ChargeSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# === GRU 模型 ===
class GRUClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]
        return self.fc(out)

# === 訓練與儲存 ===
def train_and_search_gru(batch_sizes, learning_rates, seq_lens,
                         num_epochs=100, hidden_dim=64, num_layers=1, num_classes=4, k_folds=5):

    PREPROCESSED_DIR = "./preprocessed/"

    results = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    for bs in batch_sizes:
        for lr in learning_rates:
            for seq_len in seq_lens:
                print(f"\n🧪 BS={bs} | LR={lr} | SEQ={seq_len}")
                X = np.load(os.path.join(PREPROCESSED_DIR, f"X_seq{seq_len}_3d.npy"))
                y = np.load(os.path.join(PREPROCESSED_DIR, f"y_seq{seq_len}_3d.npy"))

                skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
                acc_curves, loss_curves, f1_curves = [], [], []
                acc_train_curves, loss_train_curves = [], []
                all_y_true, all_y_pred = [], []

                t0 = time.time()

                for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                    X_train, X_val = X[train_idx], X[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]

                    train_loader = DataLoader(ChargeSequenceDataset(X_train, y_train), batch_size=bs, shuffle=True)
                    val_loader = DataLoader(ChargeSequenceDataset(X_val, y_val), batch_size=bs)

                    model = GRUClassifier(input_dim=3, hidden_dim=hidden_dim, num_layers=num_layers, num_classes=num_classes).to(device)
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                    criterion = nn.CrossEntropyLoss()

                    acc_list, loss_list, f1_list = [], [], []
                    acc_train_list, loss_train_list = [], []

                    for epoch in range(num_epochs):
                        model.train()
                        correct_train, total_train, total_loss_train = 0, 0, 0
                        for xb, yb in train_loader:
                            xb, yb = xb.to(device), yb.to(device)
                            optimizer.zero_grad()
                            out = model(xb)
                            loss = criterion(out, yb)
                            loss.backward()
                            optimizer.step()

                            _, pred = torch.max(out, 1)
                            correct_train += (pred == yb).sum().item()
                            total_train += yb.size(0)
                            total_loss_train += loss.item() * xb.size(0)

                        acc_train_list.append(correct_train / total_train)
                        loss_train_list.append(total_loss_train / total_train)

                        model.eval()
                        correct, total, total_loss = 0, 0, 0
                        y_pred_epoch, y_true_epoch = [], []
                        with torch.no_grad():
                            for xb, yb in val_loader:
                                xb, yb = xb.to(device), yb.to(device)
                                out = model(xb)
                                loss = criterion(out, yb)
                                _, pred = torch.max(out, 1)
                                correct += (pred == yb).sum().item()
                                total += yb.size(0)
                                total_loss += loss.item() * xb.size(0)
                                y_pred_epoch.extend(pred.cpu().numpy())
                                y_true_epoch.extend(yb.cpu().numpy())

                        acc_list.append(correct / total)
                        loss_list.append(total_loss / total)
                        f1_list.append(f1_score(y_true_epoch, y_pred_epoch, average='macro', zero_division=0))

                        if (epoch + 1) % 10 == 0:
                            print(f"    [Fold {fold+1}] Epoch {epoch+1}/{num_epochs} | Acc: {acc_list[-1]:.4f} | Loss: {loss_list[-1]:.4f}")

                    acc_curves.append(acc_list)
                    loss_curves.append(loss_list)
                    f1_curves.append(f1_list)
                    acc_train_curves.append(acc_train_list)
                    loss_train_curves.append(loss_train_list)
                    all_y_true.extend(y_true_epoch)
                    all_y_pred.extend(y_pred_epoch)

                t1 = time.time()
                def plot_metric(avg_list, metric_name, color, marker, folder, bs, lr, seq_len, num_epochs):
                  plt.plot(range(1, num_epochs+1), avg_list, marker=marker, color=color)
                  plt.title(f"{metric_name.capitalize()} (BS={bs}, LR={lr}, SEQ={seq_len})")
                  plt.xlabel("Epoch")
                  plt.ylabel(metric_name.capitalize())
                  plt.ylim(0, 1.0 if metric_name != 'loss' else None)
                  plt.grid(True)
                  path = os.path.join(RESULT_DIR, folder, f"bs{bs}_lr{lr}_seq{seq_len}.png")
                  plt.savefig(path, bbox_inches='tight')
                  plt.close()

                plot_metric(np.mean(acc_curves, axis=0), "accuracy", "black", "o", "accuracy_curves", bs, lr, seq_len, num_epochs)
                plot_metric(np.mean(loss_curves, axis=0), "loss", "orange", "s", "loss_curves", bs, lr, seq_len, num_epochs)
                plot_metric(np.mean(f1_curves, axis=0), "f1_score", "purple", "^", "f1_score_curves", bs, lr, seq_len, num_epochs)
                plot_metric(np.mean(acc_train_curves, axis=0), "accuracy", "red", "o", "train_accuracy", bs, lr, seq_len, num_epochs)
                plot_metric(np.mean(loss_train_curves, axis=0), "loss", "blue", "s", "train_loss", bs, lr, seq_len, num_epochs)

                cm = confusion_matrix(all_y_true, all_y_pred)
                disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
                disp.plot(cmap='Blues', values_format='d')
                plt.title(f"Confusion Matrix\nBS={bs} LR={lr} SEQ={seq_len}")
                plt.savefig(os.path.join(RESULT_DIR, "confusion_matrices", f"bs{bs}_lr{lr}_seq{seq_len}.png"), bbox_inches='tight')
                plt.close()

                final_result = {
                    'batch_size': bs, 'learning_rate': lr, 'seq_len': seq_len,
                    'final_acc': acc_list[-1], 'final_loss': loss_list[-1],
                    'precision': precision_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'recall': recall_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'f1_score': f1_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'training_time_s': round(t1 - t0, 2)
                }
                results.append(final_result)

                print(f"\n📊 統計結果 (BS={bs}, LR={lr}, SEQ={seq_len}):")
                print(f"  🔹 Final Accuracy : {final_result['final_acc']:.4f}")
                print(f"  🔹 F1-score       : {final_result['f1_score']:.4f}")
                print(f"  ⏱️  Training Time  : {final_result['training_time_s']} 秒")

    df = pd.DataFrame(results)
    csv_path = os.path.join(RESULT_DIR, "gru_experiment_results.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n📄 All experiment results saved to {csv_path}")

    if not df.empty:
        best = df.loc[df['final_acc'].idxmax()]
        print(f"\n🏆 Best: BS={best['batch_size']} | LR={best['learning_rate']} | SEQ={best['seq_len']} | ACC={best['final_acc']:.4f}")
        return results, best
    else:
        print("\n❗ No valid results")
        return [], None

In [21]:
# 設定訓練參數
batch_sizes = [4, 8, 16]
learning_rates = [1e-3, 1e-4]
seq_lens = [4, 5, 8, 10]
num_epochs = 100

# 顯示總組數
total_combinations = len(batch_sizes) * len(learning_rates) * len(seq_lens)
print(f"🔍 總共訓練組數：{total_combinations}，每組訓練 {num_epochs} epochs")

# 執行網格搜尋訓練
results, best = train_and_search_gru(batch_sizes, learning_rates, seq_lens, num_epochs=num_epochs)

# 輸出最佳參數與指標
if best is not None:
    print("\n🎯 最佳參數組合：")
    print(f"Batch Size     = {best['batch_size']}")
    print(f"Learning Rate  = {best['learning_rate']}")
    print(f"Sequence Length= {best['seq_len']}")
    print(f"Final Accuracy = {best['final_acc']:.4f}")
    print(f"Final Loss     = {best['final_loss']:.4f}")
    print(f"Precision      = {best['precision']:.4f}")
    print(f"Recall         = {best['recall']:.4f}")
    print(f"F1-score       = {best['f1_score']:.4f}")
    print(f"Training Time  = {best['training_time_s']} 秒")
else:
    print("❗ 沒有有效結果（準確率全部為 1.0）")

🔍 總共訓練組數：1，每組訓練 50 epochs

🧪 BS=16 | LR=0.0001 | SEQ=10
    [Fold 1] Epoch 10/50 | Acc: 0.9457 | Loss: 0.2989
    [Fold 1] Epoch 20/50 | Acc: 0.9444 | Loss: 0.1641
    [Fold 1] Epoch 30/50 | Acc: 0.9716 | Loss: 0.1028
    [Fold 1] Epoch 40/50 | Acc: 0.9741 | Loss: 0.0860
    [Fold 1] Epoch 50/50 | Acc: 0.9741 | Loss: 0.0800
    [Fold 2] Epoch 10/50 | Acc: 0.9432 | Loss: 0.2125
    [Fold 2] Epoch 20/50 | Acc: 0.9667 | Loss: 0.1231
    [Fold 2] Epoch 30/50 | Acc: 0.9778 | Loss: 0.0901
    [Fold 2] Epoch 40/50 | Acc: 0.9778 | Loss: 0.0795
    [Fold 2] Epoch 50/50 | Acc: 0.9827 | Loss: 0.0682
    [Fold 3] Epoch 10/50 | Acc: 0.9159 | Loss: 0.3364
    [Fold 3] Epoch 20/50 | Acc: 0.9518 | Loss: 0.1527
    [Fold 3] Epoch 30/50 | Acc: 0.9765 | Loss: 0.0945
    [Fold 3] Epoch 40/50 | Acc: 0.9790 | Loss: 0.0834
    [Fold 3] Epoch 50/50 | Acc: 0.9778 | Loss: 0.0783
    [Fold 4] Epoch 10/50 | Acc: 0.9333 | Loss: 0.2528
    [Fold 4] Epoch 20/50 | Acc: 0.9654 | Loss: 0.1483
    [Fold 4] Epoch 30/50 |

1D CNN

In [22]:
# === CNN 模型完整訓練程式 ===
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# === 資料夾與儲存路徑設定 ===
RESULT_DIR = "/content/drive/MyDrive/Colab Notebooks/test/feature dim_3/CNN"
os.makedirs(RESULT_DIR, exist_ok=True)

SUB_DIRS = [
    "accuracy_curves",
    "loss_curves",
    "f1_score_curves",
    "confusion_matrices",
    "train_accuracy",
    "train_loss",
]

for sub in SUB_DIRS:
    os.makedirs(os.path.join(RESULT_DIR, sub), exist_ok=True)

# === 自訂 Dataset ===
class ChargeSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.float32).unsqueeze(1)
        self.y = torch.tensor(labels, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# === CNN 模型 ===
class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(32, num_classes)
        )
    def forward(self, x):
        return self.model(x)

# === 主訓練與超參數搜尋函數（使用快取） ===
def train_and_search_cnn(batch_sizes, learning_rates, seq_lens,
                         num_epochs=100, num_classes=4, k_folds=5):

    PREPROCESSED_DIR = "/content/preprocessed/"
    results = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    for bs in batch_sizes:
        for lr in learning_rates:
            for seq_len in seq_lens:
                print(f"\n🧪 BS={bs} | LR={lr} | SEQ={seq_len}")
                x_path = os.path.join(PREPROCESSED_DIR, f"X_seq{seq_len}_3d.npy")
                y_path = os.path.join(PREPROCESSED_DIR, f"y_seq{seq_len}_3d.npy")
                X = np.load(x_path)
                y = np.load(y_path)

                skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

                acc_curves, loss_curves, f1_curves = [], [], []
                acc_train_curves, loss_train_curves = [], []
                all_y_true, all_y_pred = [], []

                t0 = time.time()

                for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
                    X_train, X_val = X[train_idx], X[val_idx]
                    y_train, y_val = y[train_idx], y[val_idx]

                    train_loader = DataLoader(ChargeSequenceDataset(X_train, y_train), batch_size=bs, shuffle=True)
                    val_loader = DataLoader(ChargeSequenceDataset(X_val, y_val), batch_size=bs)

                    model = CNNClassifier(num_classes=num_classes).to(device)
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                    criterion = nn.CrossEntropyLoss()

                    acc_list, loss_list, f1_list = [], [], []
                    acc_train_list, loss_train_list = [], []

                    for epoch in range(num_epochs):
                        # === Training ===
                        model.train()
                        total_loss_train, correct_train, total_train = 0, 0, 0
                        for xb, yb in train_loader:
                            xb, yb = xb.to(device), yb.to(device)
                            optimizer.zero_grad()
                            out = model(xb)
                            loss = criterion(out, yb)
                            loss.backward()
                            optimizer.step()

                            _, pred = torch.max(out, 1)
                            correct_train += (pred == yb).sum().item()
                            total_train += yb.size(0)
                            total_loss_train += loss.item() * xb.size(0)

                        acc_train = correct_train / total_train
                        loss_train = total_loss_train / total_train

                        acc_train_list.append(acc_train)
                        loss_train_list.append(loss_train)

                        # === Validation ===
                        model.eval()
                        correct, total, total_loss = 0, 0, 0
                        y_pred_epoch, y_true_epoch = [], []
                        with torch.no_grad():
                            for xb, yb in val_loader:
                                xb, yb = xb.to(device), yb.to(device)
                                out = model(xb)
                                loss = criterion(out, yb)
                                _, pred = torch.max(out, 1)
                                correct += (pred == yb).sum().item()
                                total += yb.size(0)
                                total_loss += loss.item() * xb.size(0)
                                y_pred_epoch.extend(pred.cpu().numpy())
                                y_true_epoch.extend(yb.cpu().numpy())

                        acc = correct / total
                        avg_loss = total_loss / total
                        f1 = f1_score(y_true_epoch, y_pred_epoch, average='macro', zero_division=0)

                        acc_list.append(acc)
                        loss_list.append(avg_loss)
                        f1_list.append(f1)

                        if (epoch + 1) % 10 == 0:
                            print(f"    [Fold {fold+1}] Epoch {epoch+1}/{num_epochs} | Acc: {acc:.4f} | Loss: {avg_loss:.4f}")

                    acc_curves.append(acc_list)
                    loss_curves.append(loss_list)
                    f1_curves.append(f1_list)
                    acc_train_curves.append(acc_train_list)
                    loss_train_curves.append(loss_train_list)
                    all_y_true.extend(y_true_epoch)
                    all_y_pred.extend(y_pred_epoch)

                t1 = time.time()

                def plot_metric(avg_list, metric_name, color, marker, folder):
                    plt.plot(range(1, num_epochs+1), avg_list, marker=marker, color=color)
                    plt.title(f"{metric_name.capitalize()} (BS={bs}, LR={lr}, SEQ={seq_len})")
                    plt.xlabel("Epoch")
                    plt.ylabel(metric_name.capitalize())
                    plt.ylim(0, 1.0 if metric_name != 'loss' else None)
                    plt.grid(True)
                    path = os.path.join(RESULT_DIR, folder, f"bs{bs}_lr{lr}_seq{seq_len}.png")
                    plt.savefig(path, bbox_inches='tight')
                    plt.close()

                plot_metric(np.mean(acc_curves, axis=0), "accuracy", "black", "o", "accuracy_curves")
                plot_metric(np.mean(loss_curves, axis=0), "loss", "orange", "s", "loss_curves")
                plot_metric(np.mean(f1_curves, axis=0), "f1_score", "purple", "^", "f1_score_curves")
                plot_metric(np.mean(acc_train_curves, axis=0), "accuracy", "red", "o", "train_accuracy")
                plot_metric(np.mean(loss_train_curves, axis=0), "loss", "blue", "s", "train_loss")

                cm = confusion_matrix(all_y_true, all_y_pred)
                disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
                disp.plot(cmap='Blues', values_format='d')
                plt.title(f"Confusion Matrix\nBS={bs} LR={lr} SEQ={seq_len}")
                plt.savefig(os.path.join(RESULT_DIR, "confusion_matrices", f"bs{bs}_lr{lr}_seq{seq_len}.png"), bbox_inches='tight')
                plt.close()

                final_result = {
                    'batch_size': bs, 'learning_rate': lr, 'seq_len': seq_len,
                    'final_acc': acc_list[-1], 'final_loss': loss_list[-1],
                    'precision': precision_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'recall': recall_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'f1_score': f1_score(all_y_true, all_y_pred, average='macro', zero_division=0),
                    'training_time_s': round(t1 - t0, 2)
                }
                results.append(final_result)

                print(f"\n📊 統計結果 (BS={bs}, LR={lr}, SEQ={seq_len}):")
                print(f"  🔹 Final Accuracy : {final_result['final_acc']:.4f}")
                print(f"  🔹 F1-score       : {final_result['f1_score']:.4f}")
                print(f"  ⏱️  Training Time  : {final_result['training_time_s']} 秒")

    df = pd.DataFrame(results)
    csv_path = os.path.join(RESULT_DIR, "cnn_experiment_results.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n📄 All experiment results saved to {csv_path}")

    if not df.empty:
        best = df.loc[df['final_acc'].idxmax()]
        print(f"\n🏆 Best: BS={best['batch_size']} | LR={best['learning_rate']} | SEQ={best['seq_len']} | ACC={best['final_acc']:.4f}")
        return results, best
    else:
        print("\n❗ No valid results")
        return [], None

In [23]:
# 設定訓練參數
#batch_sizes = [4, 8, 16]
#learning_rates = [1e-3, 1e-4]
#seq_lens = [4, 5, 8, 10]
#num_epochs = 100

batch_sizes = [16]
learning_rates = [1e-4]
seq_lens = [10]
num_epochs = 50


# 顯示總組數
total_combinations = len(batch_sizes) * len(learning_rates) * len(seq_lens)
print(f"🔍 總共訓練組數：{total_combinations}，每組訓練 {num_epochs} epochs")

# 執行網格搜尋訓練
results, best = train_and_search_cnn(batch_sizes, learning_rates, seq_lens, num_epochs=num_epochs)

# 輸出最佳參數與指標
if best is not None:
    print("\n🎯 最佳參數組合：")
    print(f"Batch Size     = {best['batch_size']}")
    print(f"Learning Rate  = {best['learning_rate']}")
    print(f"Sequence Length= {best['seq_len']}")
    print(f"Final Accuracy = {best['final_acc']:.4f}")
    print(f"Final Loss     = {best['final_loss']:.4f}")
    print(f"Precision      = {best['precision']:.4f}")
    print(f"Recall         = {best['recall']:.4f}")
    print(f"F1-score       = {best['f1_score']:.4f}")
    print(f"Training Time  = {best['training_time_s']} 秒")
else:
    print("❗ 沒有有效結果（準確率全部為 1.0）")

🔍 總共訓練組數：1，每組訓練 50 epochs

🧪 BS=16 | LR=0.0001 | SEQ=10
    [Fold 1] Epoch 10/50 | Acc: 0.8605 | Loss: 0.4802
    [Fold 1] Epoch 20/50 | Acc: 0.9617 | Loss: 0.2878
    [Fold 1] Epoch 30/50 | Acc: 0.9642 | Loss: 0.2157
    [Fold 1] Epoch 40/50 | Acc: 0.9667 | Loss: 0.1697
    [Fold 1] Epoch 50/50 | Acc: 0.9741 | Loss: 0.1339
    [Fold 2] Epoch 10/50 | Acc: 0.8728 | Loss: 0.4318
    [Fold 2] Epoch 20/50 | Acc: 0.9605 | Loss: 0.2157
    [Fold 2] Epoch 30/50 | Acc: 0.9704 | Loss: 0.1497
    [Fold 2] Epoch 40/50 | Acc: 0.9765 | Loss: 0.1192
    [Fold 2] Epoch 50/50 | Acc: 0.9778 | Loss: 0.1023
    [Fold 3] Epoch 10/50 | Acc: 0.8133 | Loss: 0.5794
    [Fold 3] Epoch 20/50 | Acc: 0.9333 | Loss: 0.3493
    [Fold 3] Epoch 30/50 | Acc: 0.9629 | Loss: 0.2296
    [Fold 3] Epoch 40/50 | Acc: 0.9691 | Loss: 0.1655
    [Fold 3] Epoch 50/50 | Acc: 0.9753 | Loss: 0.1307
    [Fold 4] Epoch 10/50 | Acc: 0.8195 | Loss: 0.5850
    [Fold 4] Epoch 20/50 | Acc: 0.9295 | Loss: 0.3169
    [Fold 4] Epoch 30/50 |